### Questions

### Objectives
YWBAT
- apply twitter's streamlistener to gather data on a continuous basis
- store data as csv format and save as file

### Outline

In [12]:
import json # used to read twitter data
import pandas as pd
import numpy as np
import tweepy as tw

from pprint import pprint
from pymongo import MongoClient

import matplotlib.pyplot as plt

### Load our twitter permissions and connect to Twitter using Tweepy

In [1]:
# use 'with' to open file because it automatically closes files for you
with open("") as f:
    d = json.load(f)
print(d.keys())

FileNotFoundError: [Errno 2] No such file or directory: ''

In [4]:
# create your auth
auth = tw.OAuthHandler(consumer_key=d["consumer_key"], consumer_secret=d["consumer_secret"])

In [7]:
# pass in key and secret to your auth handler
auth.set_access_token(key=d["access_token"], secret=d["access_token_secret"])

In [8]:
# create our api (the thing we use to interact with twitter) by passing in our auth
api = tw.API(auth_handler=auth)

In [17]:
# check connection with basic search (uncomment lines below to run)
# for res in api.search("Drake", tweet_mode='extended'):
#     pprint(res._json)
#     break

### Let's build a StreamListener to get more and more tweets

In [18]:
# Build our StreamListener
# Twitter requires you to populate it's methods

In [43]:
# child class of StreamListener class

class MyStreamListener(tw.StreamListener):

    def on_status(self, status):
        print(status.text)

In [48]:
# this is our listener
myStreamListener = MyStreamListener()
myStream = tw.Stream(auth = api.auth, listener=myStreamListener, tweet_mode='extended')

In [ ]:
# myStream.filter(track=["drake", "Drake", "#carepackage", "#drake", "#Drake"])

### Let's store tweets in a mongodb

### Step 1: run `mongod` in terminal and connect to it using pymongo.MongoClient

In [25]:
client = MongoClient(port=27017, host='localhost')

In [29]:
# storing names of dbs to a list
db_names_list = client.list_database_names()

In [33]:
# mongo is built on json formatting so everything reads like a dictionary
music_tweets = client["music_tweets"]

In [35]:
# new collection/table called drake
drake = music_tweets["drake"]

In [37]:
# notice we don't see the drake collection yet...why?
# this is because a collection needs a document in order to get built
music_tweets.list_collection_names()

['kendrickLamar']

### rewrite our on_status method from MyStreamListener to write into our db

In [46]:
# edited for our mongodb child class of StreamListener class

class MyStreamListener(tw.StreamListener):

    def on_status(self, status):
        j = status._json
        drake.insert_one(j)
        print(f'inserted tweet: {j["text"]}')
        pass

### Let's store tweets in our drake collection in the music_tweets db


In [ ]:
# rerun our myStream object to reset the memory
myStreamListener = MyStreamListener()
myStream = tw.Stream(auth = api.auth, listener=myStreamListener, tweet_mode='extended')

In [ ]:
myStream.filter(track=["drake", "Drake", "#carepackage", "#drake", "#Drake"])

### let's get some numbers on our new data

In [50]:
music_tweets.list_collection_names()

['drake', 'kendrickLamar']

In [52]:
drake.count_documents(filter={}) #filter={} -> 'select * from drake'

2630

In [54]:
# let's view these tweets
for res in drake.find(filter={}):
    print(res)
    break
# excuse the language

{'_id': ObjectId('5d44694751483225fdac9acc'), 'created_at': 'Fri Aug 02 16:48:02 +0000 2019', 'id': 1157332236661866500, 'id_str': '1157332236661866500', 'text': 'RT @KenTheRuthless: This nigga Drake has me hyped over songs I’ve heard before. 😪 I hate this nigga.', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 744747281547595776, 'id_str': '744747281547595776', 'name': 'Cash Marqtier 💷💵💶', 'screen_name': 'marqthemilkman', 'location': 'Sandy Springs, GA', 'url': 'http://soundcloud.com/augustmarquis', 'description': 'its like conquistador but Marquis instead 🐎Father of Fashion. Musical artist', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 136, 'friends_count': 356, 'listed_count': 0, 'favourites_count': 522

### Now let's load these into a dataframe

In [59]:
document_list = list(drake.find(filter={}))
df = pd.DataFrame(document_list) # list of jsons
df.head()

,_id,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,...,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,5d44694751483225fdac9acc,None,None,Fri Aug 02 16:48:02 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,0,0,False,{'created_at': 'Thu Aug 01 21:56:49 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @KenTheRuthless: This nigga Drake has me hy...,1564764482070,False,"{'id': 744747281547595776, 'id_str': '74474728..."
1,5d44694751483225fdac9acd,None,None,Fri Aug 02 16:48:02 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,0,0,False,{'created_at': 'Thu Aug 01 23:33:13 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @Jersey_Jinx: I'll also say that Drake's mu...,1564764482017,False,"{'id': 2847895388, 'id_str': '2847895388', 'na..."
2,5d44694751483225fdac9ace,None,None,Fri Aug 02 16:48:02 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1157042637473325056, 'id_str...",NaN,0,False,...,NaN,0,0,False,{'created_at': 'Fri Aug 02 13:46:54 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @willis_cj: bringing this gem back to the T...,1564764482155,False,"{'id': 543577551, 'id_str': '543577551', 'name..."
3,5d44694751483225fdac9acf,None,None,Fri Aug 02 16:48:02 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,0,0,False,{'created_at': 'Fri Aug 02 04:18:54 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @big_business_: Drake was giving us toxic s...,1564764482544,False,"{'id': 113987040, 'id_str': '113987040', 'name..."
4,5d44694751483225fdac9ad0,None,None,Fri Aug 02 16:48:02 +0000 2019,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,...,NaN,0,0,False,{'created_at': 'Fri Aug 02 06:07:28 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @zephaniiiah: this old drake hitting harder...,1564764482555,False,"{'id': 2516793480, 'id_str': '2516793480', 'na..."


### Assessment